# Example Thematic Map Code Usage

In [ ]:
# Import packages
from thmap.classify import ProbabilisticConcentricRandomForest
from thmap.io import ThematicMap, ImageSet
from glob import glob
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook

## Train Random Forest

### Initialize Thematic Map Object

In [ ]:
'''
Notes:  -The theme probabilities can be adjusted according to how confident the algorithm should be about determining something
            to be a feature
        -Refer to the object initialization within the code for other optional parameters that can be used in defining the object.
            For example, autofill can be turned off if you don't want a pixel to default to the most conservative theme
'''
# Define how themes are mapped to values within the data of the thematic map: standard shown below
solar_indices = {'unlabeled':0, 
                 'outer_space':1, 
                 'bright_region':3, 
                 'filament':4,
                 'prominence':5,
                 'coronal_hole':6,
                 'quiet_sun':7,
                 'limb':8,
                 'flare':9}

# Define what themes can be present in which layers of the thematic map
themes_in_radii = []
themes_in_radii.append(['bright_region', 'filament', 'coronal_hole', 'quiet_sun', 'flare'])
themes_in_radii.append(['outer_space', 'bright_region', 'prominence', 'limb', 'flare'])
themes_in_radii.append(['outer_space'])

# Define a dictionary of probabilities at which to threshold each theme
theme_probs = {'bright_region':0.85, 'filament':0.96, 'coronal_hole':0.92, 'quiet_sun':0.2, 'prominence':0.82, 'limb':0.74, 'flare':0.999, 'outer_space':0.2}

# Create a probabilistic concentric random forest object which will be used to create thematic maps
rf_classifier = ProbabilisticConcentricRandomForest(solar_indices, themes_in_radii, theme_probs)

### Train Random Forest

In [ ]:
'''
Notes:  -Training data consists of one file per chosen time containing an annotated solar image. They must be formatted 
            as fits file, and must contain both thematic map data and date in metadata 
'''

# Define path to a local directory of training data
train_dir = "/*directory_containing_train_data*"

# Define list of filenames in training data directory
train_fns = glob(train_dir + "*.fits")

# Load each file in training directory as a thematic map object
train_thmaps = [ThematicMap.load(fn) for fn in train_fns]

# Train the standard random forest model on the loaded thematic maps
rf_classifier.train(train_thmaps)

### Save Trained Random Forest

In [ ]:
'''
Notes:  -Make sure to save the model after training, otherwise an empty random forest object will be saved
'''

# Define a local path to save the random forest model to. File save format is h5
rf_save_path = "/*path_to_save_random_forest_model*"

# Use the save function on the concentric random forest object
rf_classifier.save(rf_save_path)

## Classify with Random Forest

### Load Trained Random Forest

In [ ]:
'''
Notes: *START HERE IF YOU DON'T HAVE TRAINING DATA BUT HAVE A TRAINED MODEL
        - For results similar to what is being run in real time at NOAA, use the trained model in the examples folder on github
'''

# Define a local path to save the random forest model to. File save format is h5
rf_load_path = "/*path_to_load_random_forest_model*"

# Use the save function on the concentric random forest object
rf_classifier = ProbabilisticConcentricRandomForest.load(rf_load_path)

### Classify an Image using the Random Forest

In [ ]:
'''
Notes:  -Define a date and time to for which there are composite images. The composite images will be accessed online 
            and the classifier will convert them into a thematic map. Images are taken at a 4 minute cadence
        -Classification will give an output of a thematic map object (as defined in the thematic map class)
'''
# Define the date and time to classify on. (year, month, day, hour, and minute) are necessary to define a unique image
date = datetime(2019, 5, 4, 22, 43)

# Define set of composite images corresponding to the date as an ImageSet object
image_set = ImageSet.retrieve(date)

# Use the classify method to return a thematic map object
thmap = rf_classifier.classify(image_set)

# You can also change the theme probabilities before classifying, as shown below. Note that these will be different than those used in training
theme_probs = {'bright_region':0.85, 'filament':0.96, 'coronal_hole':0.92, 'quiet_sun':0.2, 'prominence':0.82, 'limb':0.74, 'flare':0.999, 'outer_space':0.2}
rf_classifier.theme_probabilities = theme_probs

# Or you can change one individual theme by doing the following
rf_classifier.theme_probabilities['bright_region'] = 0.85

# Then classify with the new classifier object
thmap = rf_classifier.classify(image_set)

### Plot the Thematic Map

In [ ]:
'''
Notes: -The thematic map will be plotted using functionality from the thematic map class
'''

# Generate a plot of the thematic map
thmap.generate_plot()